In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial


import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
# from clarion import positions

from api.gui.base import View, Tabs
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [ ]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [ ]:
bq = BlpQuery(timeout=50000).start()
self = bq

In [ ]:
bond_ref = add_maturity_bucket(
    add_years_to_maturity(tables.bonds_reference), 
    maturity_bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 60]
)
bond_ref = add_rating_bucket(add_custom_rating(bond_ref))
bond_ref = add_geo_area(bond_ref, more_cols={'name': 'country_name'})
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [ ]:
def get_issuer_curve(bond_ref, issuer, maturities=None, currency='EUR'):
    maturities = maturities or [2, 5, 7, 10, 30, 50] 
 
    if not isinstance(maturities, list):
        maturities = [maturities]
        
    subset = bond_ref.query(f"ticker == '{issuer}'")
    if currency:   
        subset = subset.query(f"currency == '{currency}'")
    
    if subset.empty:
        return pd.DataFrame()
    
    res = []
    for maturity in maturities:
        res.append(subset.loc[(subset['years_to_workout'] - maturity).abs().idxmin()])
    return pd.concat(res, axis=1).T.squeeze().sort_values('years_to_workout').assign(maturity_label=maturities).drop_duplicates(subset='name')


def get_issuers_curves(bond_ref, maturities=None, currency=None, bmk=False):
    if not bmk:
        bond_ref = bond_ref.loc[~bond_ref['security'].isin(bond_ref['bmk_bond'])]
    return pd.concat([get_issuer_curve(bond_ref, issuer=issuer, maturities=maturities, currency=currency) 
                      for issuer in bond_ref['ticker'].drop_duplicates()])

In [ ]:
def get_interval_fields(name, field, base_name, intervals, **kwargs):
    return [
        {
            "field": f"interval_{name}",
            "name": f"{base_name}_{name}_{i}",
            "overrides": [
                ("MARKET_DATA_OVERRIDE", field),
                ("CALC_INTERVAL", i),
                *[*kwargs.items()],
            ]
        } for i in intervals
    ]

In [ ]:
def get_securities(securities, id_type=None, pcs=None, sep=None):
    id_type = id_type or ''
    if pcs:
        if id_type.lower() == 'isin':
            _sep = '@'
        else:
            _sep = ' '

        sep = sep or _sep
        securities = add_pcs(securities, source=pcs, sep=sep)

    return [f'/{id_type}/{sec}' for sec in securities] if id_type else securities

In [ ]:
def unparse_securities(securities, pcs, sep): 
    securities = pd.Series(securities)
    if pcs:
        securities = securities.str.replace(f'{sep}{pcs}', '')
    return securities.to_list()

def unparse_results_securities(res, pcs, sep):
    return res.assign(security=unparse_securities(res['security'], pcs=pcs, sep=sep))

In [ ]:
base_fields = [
    {
        "field": "blp_z_sprd_mid",
        "base_name": "z_spread_mid",
        "intervals": ["12m", "ytd", "6m", "3m", "1m", "1w", "1d"]
    },
    
    {
        "field": "yld_ytm_mid",
        "base_name": "yield_to_maturity",
        "intervals": ["12m", "ytd", "6m", "3m", "1m", "1w", "1d"]
    },
    {
        "field": "px_last",
        "base_name": "price",
        "intervals": ["12m", "ytd", "6m", "3m", "1m", "1w", "1d"]
    }

]

In [ ]:
fields = [
    {
        "field": "BLP_Z_SPRD_MID",
        "name": "z_spread_mid",
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_12m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "12m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_ytd",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "YTD")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_6m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "6m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_3m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "3m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_1m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "1m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "z_spread_change_1w",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "BLP_Z_SPRD_MID"),
            ("CALC_INTERVAL", "1w")
        ]
    },

    {
        "field": "yld_ytm_mid",
        "name": "yield_to_maturity_mid",
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_12m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "12m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_ytd",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "YTD")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_6m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "6m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_3m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "3m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_1m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "1m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "yield_to_maturity_change_1w",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "YLD_YTM_MID"),
            ("CALC_INTERVAL", "1w")
        ]
    },

    {
        "field": "px_last",
        "name": "price",
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_12m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "12m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_ytd",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "YTD")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_6m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "6m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_3m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "3m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_1m",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "1m")
        ]
    },
    {
        "field": "INTERVAL_NET_CHANGE",
        "name": "price_change_1w",
        "overrides": [
            ("MARKET_DATA_OVERRIDE", "px_last"),
            ("CALC_INTERVAL", "1w")
        ]
    },
]

In [ ]:
issuers_curves = {cur: get_issuers_curves(bond_ref, currency=cur) for cur in bond_ref['currency'].drop_duplicates()}

In [ ]:
fields_mapping = {
    "blp_z_sprd_mid": "z_spread_mid",
    "yld_ytm_mid": "yield_to_maturity",
    "px_last": "price"
}

In [ ]:
class BBGDownload(View):
    def __init__(self, fields_mapping=None, **kwargs):
        super().__init__(**kwargs)
        self.fields_mapping = fields_mapping or {}
    
    def make_widgets(self):
        pcs_tf = v.TextField(
            v_model='BGN',
            label='pcs',
            clearable=True,
            dense=True,
            outlined=True,

        )
        
        sep_tf = v.TextField(
            v_model='@',
            label='sep',
            clearable=True,
            dense=True,
            outlined=True,
            disabled=True
        )
        
        interval_cb = v.Combobox(
            v_model=["12m", "ytd", "6m", "3m", "1m", "1w", "2d"],
            items=["12m", "ytd", "6m", "3m", "1m", "1w", "2d"],
            multiple=True,
            small_chips=True,
            deletable_chips=True,
            outlined=True,
            dense=True,
            clearable=True,
            label="fields"
        )
        
        fields_cb = v.Combobox(
            v_model=["blp_z_sprd_mid", "yld_ytm_mid", "px_last"],
            items=["blp_z_sprd_mid", "yld_ytm_mid", "px_last"],
            multiple=True,
            small_chips=True,
            deletable_chips=True,
            outlined=True,
            dense=True,
            clearable=True,
            label="intervals"
        )
        
        dl_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-download'])]
        )
        
        self.pcs_tf = pcs_tf
        self.sep_tf = sep_tf
        self.interval_cb = interval_cb
        self.fields_cb = fields_cb
        self.dl_btn = dl_btn
        self.out = w.Output()
        
    def make_view(self, **kwargs):
        pcs_tf = self.pcs_tf
        sep_tf = self.sep_tf 
        interval_cb = self.interval_cb
        fields_cb = self.fields_cb
        dl_btn = self.dl_btn
        out= self.out
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=3,
                            children=[pcs_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=3,
                            children=[sep_tf],
                            class_="my-0 py-0"
                        ),
                    ]
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=8,
                            children=[fields_cb],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=8,
                            children=[interval_cb],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[dl_btn],
                            class_="my-0 py-0"
                        ),
                    ]
                ),
            ]
        )

        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
        
    def make_fields(self):
        return [*chain.from_iterable([
            [
                {"field": field, "name": self.fields_mapping.get(field, field)}, 
                *get_interval_fields(
                    name='net_change', 
                    field=field, 
                    base_name=self.fields_mapping.get(field, field),
                    intervals=self.interval_cb.v_model,
                )
            ] for field in self.fields_cb.v_model])]
    
    def get_bbg_securities(self):
        securities = pd.concat([value['security'] for key, value in issuers_curves.items()]).drop_duplicates()
        securities = get_securities(securities, pcs=self.pcs_tf.v_model, sep=self.sep_tf.v_model)
        return securities
    
    def get_bbg_fields(self):
        fields = self.make_fields()
        bbg_fields = [
            BBGField(
                field=field['field'], 
                name=field['name'], 
                overrides=BBGOverrides(**dict(field['overrides'])) if field.get('overrides') else None
            ) for field in fields
        ]
        return bbg_fields
    
    def download(self):
        securities = self.get_bbg_securities()
        bbg_fields = self.get_bbg_fields()
        mkt = bq.bdpy(securities=securities, bbg_fields=bbg_fields)
        mkt = unparse_results_securities(mkt, pcs=self.pcs_tf.v_model, sep=self.sep_tf.v_model)
        self.mkt = mkt
        self.store.data['mkt'] = mkt
        try:
            self.store.tabs['pivot'].value_sel.items = [*mkt.columns[1:]]
        except Exception as e:
            display(e)
        
    def link(self, **kwargs):
        dl_btn = self.dl_btn
        out = self.out
        
        dl_btn.on_event(
            'click',
            partial(
                out.capture(clear_output=True)(on_click_download),
                self=self
            )
        )

In [ ]:
def on_click_download(widget, event, payload, self):
    widget.loading = True
    try:
        self.download()
    except Exception as e:
        widget.color = 'error'
    else:
        widget.color = 'success'
    finally:
        widget.loading = False

In [ ]:
self = BBGDownload(fields_mapping=fields_mapping)
self

In [ ]:
class CurvesViewer(View):
    def make_widgets(self):
        curr_sel = v.Autocomplete(
            v_model='USD',
            items=[*issuers_curves],
            label='currency',
            dense=True,
            outlined=True
        )
        self.curr_sel = curr_sel
        self.out = w.Output()
        
    def make_view(self):
        curr_sel = self.curr_sel
        self.param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=3,
                            children=[curr_sel],
                            class_="my-0 py-0"
                        ),
                    ]
                )
            ]
        )
        
        self.view = w.VBox(
            [
                self.param_box, 
                self.out
            ]
        )
        
    def link(self):
        curr_sel = self.curr_sel
        out = self.out
        curr_sel.on_event(
            'change',
            partial(
                out.capture(clear_output=True)(on_change_dg),
                self=self
            )
        )
        curr_sel.fire_event('change', None)

In [ ]:
def on_change_dg(widget, event, data, self):
    dg = DataGrid(issuers_curves[self.curr_sel.v_model].set_index('name'), selection_mode='cell')
    dg.auto_fit_columns = True
    display(dg)

In [ ]:
def make_datagrid(data, field=None):
    field = field or ''
    renderers = {
        col: BarRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
            horizontal_alignment="center",
            # bar_horizontal_alignment="center",
            bar_color=ColorScale(min=data[col].quantile(0.1), mid=0 if 'change' in field else None,  max=data[col].quantile(0.9)),
            bar_value=LinearScale(min=data[col].quantile(0.1), mid=0 if 'change' in field else None, max=data[col].quantile(0.9))
        ) for col in data
    } 
    
    dg = DataGrid(
        data, 
        renderers=renderers, 
        editable=False,
        selection_mode='cell',
        base_column_size=100,
    )
    # dg.auto_fit_columns = True
    return dg


In [ ]:
def get_pivot(df, field, index, columns, filters=None):
    if filters:
        q = ' and '.join([f"{key} == {value}" for key, value in filters.items()])
        df = df.query(q)

    return df.pivot_table(
        values=field,
        index=index,
        columns=columns
    )

In [ ]:
def make_slopes_df(pivot_df, slopes=None):
    slopes = slopes or [
        (30, 10),
        (10, 5),
    ]

    slope_series = []
    for slope in slopes:
        s = pivot_df[slope[0]] - pivot_df[slope[1]]
        slope_series.append(s.rename(f"{slope[0]} - {slope[1]}"))

    return pd.concat(slope_series, axis=1).dropna(how='all')

In [ ]:
def on_click_make_pivot(widgets, event, payload, self):
    # data = get_issuers_curves(bond_ref, currency=self.curr_sel.v_model)
    # data = data.merge(mkt, on='security', how='left')
    
    data = self.get_data()

    pivot_df = get_pivot(
        data,
        self.value_sel.v_model,
        self.index_ac.v_model,
        self.cols_ac.v_model
    )
    dg = make_datagrid(pivot_df, self.value_sel.v_model)
    self._pivot = pivot_df

    # display(dg)

    slopes_df = make_slopes_df(pivot_df)
    dg2 = make_datagrid(slopes_df)

    # display(box)

    box = w.HBox([dg, dg2])
    display(box)

In [ ]:
class Pivot(View):
    def get_data(self):
        try:
            mkt = self.store.data['mkt']
        except KeyError:
            mkt = mkt
        
        data = issuers_curves[self.curr_sel.v_model]
        data = data.merge(mkt, on='security', how='left')
        return data
    
    def make_widgets(self):
        curr_sel = v.Autocomplete(
            v_model='USD',
            items=[*issuers_curves.keys()],
            label='currency',
            dense=True,
            outlined=True
        )

        value_sel = v.Autocomplete(
            v_model='z_spread',
            # items=[field.name for field in bbg_fields],
            label='field',
            dense=True,
            outlined=True
        )

        index_ac = v.Autocomplete(
            v_model=['ticker'],
            label='index',
            items=[
                'ticker',
                'country_name',
                'region',
                'subregion',
                'rating_bucket',
            ],
            outlined=True,
            multiple=True,
            chips=True,
            dense=True,
            clearable=True,
            # style_='width: 250px',
            # class_='ma-0 pa-0'
        )

        cols_ac = v.Autocomplete(
            v_model='maturity_label',
            label='cols',
            items=['maturity_label', 'rating_bucket'],
            outlined=True,
            # multiple=True,
            chips=True,
            dense=True,
            clearable=True,
            # style_='width: 250px',
            # class_='ma-0 pa-0'
        )

        table_btn = v.Btn(
            fab=True,
            small=True,
            plain=True,
            left=True,
            children=[v.Icon(children=['mdi-table'])]
        )

        out = w.Output()

        self.curr_sel = curr_sel
        self.value_sel = value_sel
        self.index_ac = index_ac
        self.cols_ac = cols_ac
        self.table_btn = table_btn
        self.out = out

    def make_view(self):
        curr_sel = self.curr_sel
        value_sel = self.value_sel
        index_ac = self.index_ac
        cols_ac = self.cols_ac
        table_btn = self.table_btn
        out = self.out

        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        
                        v.Col(
                            cols=3,
                            children=[curr_sel],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=3,
                            children=[value_sel],
                            class_="my-0 py-0"
                        ),
                    ]
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=3,
                            children=[index_ac],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=3,
                            children=[cols_ac],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[table_btn],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
            ]
        )

        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view

    def link(self):
        # curr_sel = self.curr_sel
        value_sel = self.value_sel
        index_ac = self.index_ac
        cols_ac = self.cols_ac
        table_btn = self.table_btn
        out = self.out

        table_btn.on_event(
            'click',
            partial(
                out.capture(clear_output=True)(on_click_make_pivot),
                self=self
            ) 
        )

In [285]:
app = Tabs(
    {
        'curves': CurvesViewer(build=False),
        'bbg': BBGDownload(fields_mapping=fields_mapping, build=False),
        'pivot': Pivot(build=False)
    }
)

In [286]:
app

Card(children=[Tabs(children=[Tab(children=['curves']), Tab(children=['bbg']), Tab(children=['pivot'])], v_mod…

In [106]:
self = Pivot()
self

In [109]:
res = bq.bdh(
    securities=securities,
    fields=[
        'blp_z_sprd_mid',
        'yld_ytm_mid',
        'px_last',
    ],
    start_date='20200101',
    end_date='20220720'
)

In [111]:
res.pivot(index='date', columns='security')

blp_z_sprd_mid                                      \
security   AM361562@BGN Corp AM515268@BGN Corp AM752854@BGN Corp   
date                                                               
2020-01-01               NaN               NaN               NaN   
2020-01-02           588.522           217.017            75.659   
2020-01-03           593.384           219.317            81.656   
2020-01-06           596.904           219.594            80.854   
2020-01-07           571.459           219.475            80.533   
...                      ...               ...               ...   
2022-07-14          5252.571           474.698           114.488   
2022-07-15          5260.351           470.393           121.508   
2022-07-18          5278.562           467.987           115.410   
2022-07-19          5412.726           456.766           111.076   
2022-07-20          5125.545           454.769           105.242   

                                                                  \
security   AM837814@BGN Corp AN206370@BGN Corp AN649465@BGN Corp   
date                                                               
2020-01-01            52.810            98.871             5.204   
2020-01-02            57.319           103.092             9.274   
2020-01-03            58.797           105.493            12.622   
2020-01-06            58.978           104.893            12.377   
2020-01-07            59.262           105.360            13.446   
...                      ...               ...               ...   
2022-07-14            63.298           348.529            84.360   
2022-07-15            66.711           348.857            84.631   
2022-07-18            61.630           345.344            80.209   
2022-07-19            57.811           339.710            77.877   
2022-07-20            58.079           337.276            77.904   

                                                                  \
security   AP305701@BGN Corp AQ927618@BGN Corp AS686996@BGN Corp   
date                                                               
2020-01-01            23.175               NaN               NaN   
2020-01-02            27.268            41.886           381.201   
2020-01-03            29.354            45.920           385.046   
2020-01-06            29.370            45.417           385.926   
2020-01-07            29.649            45.986           385.239   
...                      ...               ...               ...   
2022-07-14           207.803           179.054           833.060   
2022-07-15           208.944           180.234           833.332   
2022-07-18           200.111           177.270           825.703   
2022-07-19           196.182           170.811           817.128   
2022-07-20               NaN           167.548           815.256   

                              ...           px_last                    \
security   AS959239@BGN Corp  ... ZO496420@BGN Corp ZO496428@BGN Corp   
date                          ...                                       
2020-01-01               NaN  ...               NaN               NaN   
2020-01-02            60.905  ...               NaN               NaN   
2020-01-03            62.286  ...               NaN               NaN   
2020-01-06            63.182  ...               NaN               NaN   
2020-01-07            63.046  ...               NaN               NaN   
...                      ...  ...               ...               ...   
2022-07-14           102.952  ...            76.818            59.631   
2022-07-15           104.052  ...            76.967            59.710   
2022-07-18           105.895  ...            76.952            59.924   
2022-07-19           104.222  ...            76.914            60.038   
2022-07-20           108.374  ...            77.169            60.023   

                                                                  \
security   ZO640831@BGN Corp ZP606054@BGN Corp ZQ466636@BGN Corp